# 식신 - 제주시청

In [1]:
# 식신 - 제주시청

In [2]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

### 1. 데이터 가져오기

In [3]:
site = 'https://www.siksinhot.com'
url = f'{site}/search?keywords={quote("제주시청")}'
req = requests.get(url)
html = req.text
type(html)

str

In [4]:
soup = BeautifulSoup(html,'html.parser')

In [5]:
# type(soup)

## # 2. 찾으려는 하는 데이터의 태그 찾기

In [6]:
# lis = soup.select_one('div.listTy1').find_all('li') 
# len(lis)

In [7]:
lis = soup.select('div.listTy1> ul > li') # 자식 Selector
len(lis)

12

### 3. 하나를 선택해서 원하는 정보 추출
- 상호, 평점, 메뉴 , 서브 페이지의 url
- 전화번호, 주소

In [8]:
li = lis[1]
href = li.find('a')['href']


In [9]:
score = li.select_one(".score").string
score

'3.4'

In [10]:
store = li.select_one(".store").string
store

'안양돈까스 제주시점'

In [11]:
menu = li.find('p').get_text()
menu

'돈까스 세트, 치즈돈까스 세트'

In [12]:
score,store,menu,href

('3.4', '안양돈까스 제주시점', '돈까스 세트, 치즈돈까스 세트', '/P/466268')

In [13]:
sub_url = site + href
sub_url


'https://www.siksinhot.com/P/466268'

In [14]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [15]:
tel = sub_soup.select_one('.p_tel').find('p').get_text()
tel

'064-799-1434'

In [16]:
addr = sub_soup.select_one('.txt_adr').get_text()
addr

'제주특별자치도 제주시 애월읍 애월로 40'

In [19]:
# try - except 를 이용해서 어디에서 에러가 발생하는지 확인하고 수정하기 
lines=[]
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one(".score").string
        store = li.select_one(".store").get_text()
        menu = li.find('p').get_text()
        
        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except:
        print(index)
    
    lines.append([store,score,menu,tel,addr])


0


### 5. 에러난 부분을 처리할 수 있도록 수정

In [27]:
li = lis[0]
score = li.select_one('.cnt>em').get_text()
type(score)

str

In [28]:
li = lis[1]
score = li.select_one('.cnt>em').get_text()
type(score)

str

### 6. 에러가 해결될 때 까지 5, 6을 반복

In [48]:
lines=[]
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.cnt>em').string
        if score == '평가중':
            score = None
        store = li.select_one(".store").string
        menu = li.find('p').get_text()
        
        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    
    except:
        print(index)
    
    lines.append([store,score,menu,tel,addr])

In [49]:
lines

[['제주시새우리',
  None,
  '딱새우김밥, 간장새우김밥, 양념새우컵밥, 딱새우꼬막무침, 한라봉에이드',
  '064-900-2527',
  '제주특별자치도 제주시 무근성7길 21'],
 ['안양돈까스 제주시점',
  '3.4',
  '돈까스 세트, 치즈돈까스 세트',
  '064-799-1434',
  '제주특별자치도 제주시 애월읍 애월로 40'],
 ['올래국수', '3.8', '고기국수', '064-742-7355', '제주특별자치도 제주시 귀아랑길 24'],
 ['명진전복',
  '3.8',
  '전복죽, 전복돌솥밥, 전복구이, 전복회',
  '064-782-9944',
  '제주특별자치도 제주시 구좌읍 해맞이해안로 1282'],
 ['삼성혈해물탕',
  '3.9',
  '해물탕(특대), 해물탕(대), 해물탕(중), 해물탕(소), 전복회(소)',
  '064-745-3000',
  '제주특별자치도 제주시 선덕로5길 20'],
 ['자매국수',
  '3.9',
  '멸치국수, 고기국수, 비빔국수, 멸/고기국수, 아강발(미니족박), 콩국수',
  '064-746-2222',
  '제주특별자치도 제주시 탑동로11길 6 '],
 ['해녀촌', '3.7', '회국수, 성게국수, 소라국수', '064-783-5438', '제주특별자치도 제주시 구좌읍 동복로 33'],
 ['순옥이네명가',
  '3.8',
  '소라, 성게, 한치물회, 순옥이네물회, 전복물회',
  '064-743-4813',
  '제주특별자치도 제주시 도공로 8'],
 ['은희네해장국', '3.9', '소고기해장국, 해장국, 내장탕', '064-726-5622', ''],
 ['삼대국수회관 본점',
  '3.9',
  '고기국수, 비빔국수, 돔베고기 세트, 물만두 세트',
  '064-759-6645',
  '제주특별자치도 제주시 삼성로 41'],
 ['다가미',
  '4.1',
  '다가미김밥, 소고기롤, 단호박 샌드위치, 참치로얄, 매운 멸치쌈',
  '064-758-5810',
  

### 7. 데이터 프레임으로 만들기

In [50]:
df = pd.DataFrame(lines, columns=['업소명','평점','메뉴','전화번호','주소'])
df

,업소명,평점,메뉴,전화번호,주소
0,제주시새우리,None,"딱새우김밥, 간장새우김밥, 양념새우컵밥, 딱새우꼬막무침, 한라봉에이드",064-900-2527,제주특별자치도 제주시 무근성7길 21
1,안양돈까스 제주시점,3.4,"돈까스 세트, 치즈돈까스 세트",064-799-1434,제주특별자치도 제주시 애월읍 애월로 40
2,올래국수,3.8,고기국수,064-742-7355,제주특별자치도 제주시 귀아랑길 24
3,명진전복,3.8,"전복죽, 전복돌솥밥, 전복구이, 전복회",064-782-9944,제주특별자치도 제주시 구좌읍 해맞이해안로 1282
4,삼성혈해물탕,3.9,"해물탕(특대), 해물탕(대), 해물탕(중), 해물탕(소), 전복회(소)",064-745-3000,제주특별자치도 제주시 선덕로5길 20
5,자매국수,3.9,"멸치국수, 고기국수, 비빔국수, 멸/고기국수, 아강발(미니족박), 콩국수",064-746-2222,제주특별자치도 제주시 탑동로11길 6
6,해녀촌,3.7,"회국수, 성게국수, 소라국수",064-783-5438,제주특별자치도 제주시 구좌읍 동복로 33
7,순옥이네명가,3.8,"소라, 성게, 한치물회, 순옥이네물회, 전복물회",064-743-4813,제주특별자치도 제주시 도공로 8
8,은희네해장국,3.9,"소고기해장국, 해장국, 내장탕",064-726-5622,
9,삼대국수회관 본점,3.9,"고기국수, 비빔국수, 돔베고기 세트, 물만두 세트",064-759-6645,제주특별자치도 제주시 삼성로 41


In [43]:
import seaborn as s


